In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 
import torchvision  
import skorch 
import sklearn 
import numpy as np
import matplotlib.pyplot  as plt 

"""
This file runs a basic CNN and a max pooling CNN on the CIFAR-10 data set. It outputs statistics including validation accuracy for different settings of epochs, channels, learning rate, and optimizer.

CIFAR-10 dataset:
    - each image has a size of 32 x 32 pixels
    - image dataset of 10 categories

Sources:
    - https://www.datacamp.com/tutorial/pytorch-cnn-tutorial
    - https://skorch.readthedocs.io/en/stable/classifier.html#skorch.classifier.NeuralNetClassifier
"""

# make datasets
train = torchvision.datasets.CIFAR10("./data", train=True, download=True) 
test = torchvision.datasets.CIFAR10("./data", train=False, download=True)


In [72]:
"""
Basic CNN implementation
    - 3 convolutional layers folowed by 1 linear layer
    - each convolutional layer has a kernel size of 3, padding of 1
    - ReLU activation is applied on every hidden layer
"""
class CNN(nn.Module):
    def __init__(self, channels, num_classes):
        super(CNN, self).__init__()
        # implement parameter definitions here
        tuned_channels = 128
        self.conv1 = nn.Conv2d(in_channels = channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(in_channels = tuned_channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(in_channels = tuned_channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)

        self.linear_layer = nn.Linear(tuned_channels * 32 * 32, num_classes)


    def forward(self, images):
        # implement the forward function here
        images = F.relu(self.conv1(images))
        images = F.relu(self.conv2(images))
        images = F.relu(self.conv3(images))
        images = images.reshape(images.shape[0], -1)
        images = self.linear_layer(images)

        return images

In [73]:
"""
Max Pool CNN Implementation
    - 3 convolution layers with max pool after each
    - kernel size of 2, stride of 2
    - ReLU activation applied on hidden layers
"""
class CNN_MaxPool(nn.Module):
    def __init__ (self, channels, num_classes): 
        super(CNN_MaxPool, self).__init__() 
        # implement parameter definitions here 
        tuned_channels = 128
        self.conv1 = nn.Conv2d(in_channels = channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv2 = nn.Conv2d(in_channels = tuned_channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv3 = nn.Conv2d(in_channels = tuned_channels, out_channels = tuned_channels, kernel_size = 3, padding = 1)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.linear_layer = nn.Linear(tuned_channels * 4 * 4, num_classes)

    def forward (self, images) : 
        # implement the forward function here 
        images = F.relu(self.conv1(images))
        images = self.pool(images)
        images = F.relu(self.conv2(images))
        images = self.pool(images)
        images = F.relu(self.conv3(images))
        images = self.pool(images)
        images = images.reshape(images.shape[0], -1)
        images = self.linear_layer(images)
        
        return images

In [ ]:
# implement hyperparameters here 
num_classes = 10
num_channels = 3
# tune the hyperparameters here
learning_rate = 0.0005
num_epochs = 5

"""
The optimizer is currently set to Adam. To switch to SGD, change the optimizer parameter to optim.SGD
"""
model1 = skorch.NeuralNetClassifier(CNN, 
                                   module__channels = num_channels, 
                                   module__num_classes = num_classes, 
                                   criterion = torch.nn.CrossEntropyLoss,
                                   optimizer = optim.Adam,
                                   lr = learning_rate,
                                   max_epochs = num_epochs,
                                   verbose = 1,
                                   device = "cuda") 

"""
The optimizer is currently set to Adam. To switch to SGD, change the optimizer parameter to optim.SGD
"""

model2 = skorch.NeuralNetClassifier(CNN_MaxPool, 
                                   module__channels = num_channels, 
                                   module__num_classes = num_classes, 
                                   criterion = torch.nn.CrossEntropyLoss,
                                   optimizer = optim.Adam,
                                   lr = learning_rate,
                                   max_epochs = num_epochs,
                                   verbose = 1,
                                   device = "cuda") 

# implement input normalization & type cast here 
# print(train.data.shape)

# get targets as numpy array
train.targets = np.array(train.targets, dtype=np.int64)

# get data in correct order for fit function
train.data = train.data.transpose(0,3,1,2)
# standardize between 0 and 1
train.data = train.data.astype(np.float32) / np.float32(255.0)

# get means and standard deviations
means = train.data.mean(axis=(0, 2, 3), dtype=np.float32) 
stds  = train.data.std (axis=(0, 2, 3), dtype=np.float32) 

# normalize data by subtracting means and dividing by stds
train.data = ((train.data - means[None, :, None, None]) / stds[None, :, None, None])

# testing info
print("Learning Rate:", learning_rate)
print("Num Epochs:", num_epochs)

# fit to data
print("Basic CNN:")
model1.fit(train.data, train.targets)  
print("\nFull CNN:")
model2.fit(train.data, train.targets)


(50000, 32, 32, 3)
Learning Rate: 0.0005
Num Epochs: 5
Basic CNN:
  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        1.5095       0.5648        1.2296  24.5340
      2        1.1503       0.6177        1.0766  22.6760
      3        0.9523       0.6463        0.9992  22.7199
      4        0.7811       0.6604        1.0025  22.7333
      5        0.6240       0.6492        1.1003  22.8761
Full CNN:
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        1.5506       0.5526        1.2635  8.6599
      2        1.1332       0.6413        1.0306  8.5537
      3        0.9384       0.6859        0.9064  8.5654
      4        0.8208       0.7112        0.8265  9.2741
      5        0.7357       0.7219        0.7989  9.5843


,module,<class '__main__.CNN_MaxPool'>
,criterion,<class 'torch...sEntropyLoss'>
,train_split,<skorch.datas...x711a9986dd80>
,classes,None
,optimizer,<class 'torch...im.adam.Adam'>
,lr,0.0005
,max_epochs,5
,batch_size,128
,iterator_train,<class 'torch...r.DataLoader'>
,iterator_valid,<class 'torch...r.DataLoader'>
,dataset,<class 'skorc...aset.Dataset'>
